In [23]:
import gensim
import numpy as np
import pandas as pd
import scipy.stats as st

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_distances

In [24]:
!head -n 4 '../data/exp1-trj.t' 

508 465 1641 857 3176 1346 1301 3303 3277 3977 4430 8513 9755 11383 9496 12228 11150 13279 9215 17279 14428 9279 14792 14310 18351 7997 15024 15267 15665 16329 15125 14591 14797 3
19 191 68 41 46 4 964 543 154 171 382 732 632 923 4815 460 273 439 607 908 726 6277 5338 5821 6933 5438 8005 10438 9812 17030 12351 12132 13988 9320 5581 13142 15720 14944 5048 11012
51 2263 345 53 120 405 803 585 692 1566 533 1728 1880 739 544 226 8
15 4 58 9 38 54 196 648 193 1022 575 1560 753 229 260 532 317 764 159 633 1304 207 798 2811 301 211 880 80 23 89 40 537 781 141 5900 240 38 54 655 1616 196 648 437 749 575


In [25]:
with open('../data/exp1-trj.t', 'r') as file:
    trajs_teste = []

    for line in file:
        traj = line.strip().split()# Divide cada linha(traj) em cels usando espaço como delimitador e convertendo-as em strings
        trajs_teste.append(traj)

In [26]:
print("Quantidade de trajetórias de teste:", len(trajs_teste))

Quantidade de trajetórias de teste: 101000


In [27]:
print(trajs_teste[2]) # query par

['51', '2263', '345', '53', '120', '405', '803', '585', '692', '1566', '533', '1728', '1880', '739', '544', '226', '8']


In [28]:
print(trajs_teste[1002]) # "alvo" da query par, ou seja, a query ímpar

['51', '430', '345', '120', '856', '131', '673', '585', '233', '2200', '533', '361', '1299', '1215', '66', '588', '7', '8']


In [29]:
!ls '../data/saved_models/W2V/'

w2v_w5_s128_ep5_sg1.model  w2v_w5_s64_ep1_sg1.model
w2v_w5_s256_ep3_sg1.model  w2v_w5_s64_ep3_sg1.model


In [30]:
# Carregue o modelo que prentende testar...
model = Word2Vec.load('../data/saved_models/W2V/w2v_w5_s256_ep3_sg1.model')

In [31]:
# size vocab
len(model.wv)

18827

In [32]:
model.vector_size

256

In [33]:
def get_embedding_mean_for_all_trajs(trajs):
    # Vetores de embeddigns médios
    emb_trajs = np.zeros((len(trajs),) + (model.vector_size,), dtype=np.float32) # shape --> (len(trajs), vector_size). Ex. para Dq, shape --> (1000, 64)

    idx = 0
    for traj in trajs:
        vet_sum = np.zeros(model.vector_size, dtype='float32')
        for i in range(len(traj)):
          try:
            vet_sum += model.wv.get_vector(traj[i])
          except KeyError:
            pass # Se a palavra não existe no vocab, assume-se ela como sendo um vetor de zeros!
        emb_trajs[idx] = vet_sum/len(traj)
        idx += 1

        # Calcula a porcentagem concluída
        percent_done = (idx / len(trajs)) * 100
        print(f"Progresso: {percent_done:.2f}% concluído", end="\r")  # A opção `end="\r"` permite que a impressão seja substituída na mesma linha


    return emb_trajs

In [34]:
# Segmentando: query (trajs pares) e dbsearch (querys ímpar + 99000 outras ímpares)
query = trajs_teste[:1000] # trajs query (pares)
dbsearch = trajs_teste[1000:101000] # dbsearch trajs (as 1000 primeiras são as query ímpar)

In [35]:
print(len(query))
print(len(dbsearch))

1000
100000


In [36]:
# Pegando os embeddings das trajs query:
query = get_embedding_mean_for_all_trajs(query)

In [37]:
# Pegando os embeddings das trajs do dbsearch:
dbsearch = get_embedding_mean_for_all_trajs(dbsearch)

In [38]:
def rank(t_query, i, db_search): 
    dists = cosine_distances(t_query.reshape(1, -1), db_search) # pega todas as distâncias de Dq[i] as trajs do db_search (D_qUD_p)
    dists = dists.flatten()
    order = dists.argsort() # pega a ordem
    ranks = order.argsort() # pega o rank

    return ranks[i] + 1 # retorna o ranking de ta' no dbsearch | soma +1 pq o argsort rankea a partir de 0 (zero)

In [39]:
def acc(ranks):
    count = 0
    for i in range(len(ranks)):
        if ranks[i] == 1:
            count += 1

    return round(count/len(ranks), 3)

In [40]:
def mr(ranks):
    return (sum(ranks)/len(ranks))

In [41]:
def mrr(ranks):
    count = 0
    for i in range(len(ranks)):
        count += 1/ranks[i]

    return round(count/len(ranks), 3)

In [42]:
# Intervalo de Confiança do Ranks
def cip_r(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(ranks[i]) # Add os Ranks
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [43]:
# Intervalo de Confiança dos Reciprocal Ranks
def cip_rr(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(1/ranks[i]) # Add os RRs...
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [44]:
%%time
# w2v_w5_s256_ep5_sg1
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    ranks = []
    search = dbsearch[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(query[i], i, search)) # rank das ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 2.978(2.343, 3.613), Acc: 0.856, MRR: 0.896(0.88, 0.912) with dbsearch size: 20000
Mean rank: 4.632(3.455, 5.809), Acc: 0.819, MRR: 0.869(0.851, 0.887) with dbsearch size: 40000
Mean rank: 6.344(4.585, 8.103), Acc: 0.789, MRR: 0.847(0.828, 0.866) with dbsearch size: 60000
Mean rank: 8.729(6.15, 11.308), Acc: 0.769, MRR: 0.829(0.808, 0.849) with dbsearch size: 80000
Mean rank: 10.567(7.381, 13.753), Acc: 0.759, MRR: 0.818(0.797, 0.839) with dbsearch size: 100000
CPU times: user 15min 20s, sys: 37min 50s, total: 53min 10s
Wall time: 13min 39s


In [22]:
%%time
# w2v_w5_s128_ep5_sg1
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    ranks = []
    search = dbsearch[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(query[i], i, search)) # rank das ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 2.95(2.3, 3.6), Acc: 0.86, MRR: 0.901(0.885, 0.917) with dbsearch size: 20000
Mean rank: 4.6(3.402, 5.798), Acc: 0.825, MRR: 0.873(0.855, 0.891) with dbsearch size: 40000
Mean rank: 6.277(4.49, 8.064), Acc: 0.8, MRR: 0.855(0.836, 0.874) with dbsearch size: 60000
Mean rank: 8.694(6.063, 11.325), Acc: 0.776, MRR: 0.836(0.816, 0.856) with dbsearch size: 80000
Mean rank: 10.532(7.285, 13.779), Acc: 0.765, MRR: 0.826(0.805, 0.846) with dbsearch size: 100000
CPU times: user 13min 36s, sys: 32min 16s, total: 45min 53s
Wall time: 13min 42s


In [26]:
%%time
# w2v_w5_s64_ep1_sg1
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    ranks = []
    search = dbsearch[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(query[i], i, search)) # rank das ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 2.387(1.892, 2.882), Acc: 0.883, MRR: 0.915(0.9, 0.93) with dbsearch size: 20000
Mean rank: 3.616(2.657, 4.575), Acc: 0.854, MRR: 0.893(0.876, 0.91) with dbsearch size: 40000
Mean rank: 4.911(3.447, 6.375), Acc: 0.826, MRR: 0.873(0.855, 0.891) with dbsearch size: 60000
Mean rank: 6.663(4.523, 8.803), Acc: 0.797, MRR: 0.852(0.833, 0.871) with dbsearch size: 80000
Mean rank: 7.975(5.335, 10.615), Acc: 0.787, MRR: 0.843(0.823, 0.862) with dbsearch size: 100000
CPU times: user 10min 39s, sys: 30min 20s, total: 40min 59s
Wall time: 10min 19s


In [22]:
%%time
# w2v_w5_s64_ep3_sg1
dbsizes = [20000, 40000, 60000, 80000, 100000]
for dbsize in dbsizes:
    ranks = []
    search = dbsearch[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(query[i], i, search)) # rank das ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 2.576(2.032, 3.12), Acc: 0.88, MRR: 0.913(0.898, 0.929) with dbsearch size: 20000
Mean rank: 3.901(2.896, 4.906), Acc: 0.85, MRR: 0.89(0.873, 0.907) with dbsearch size: 40000
Mean rank: 5.316(3.802, 6.83), Acc: 0.823, MRR: 0.87(0.851, 0.888) with dbsearch size: 60000
Mean rank: 7.245(5.039, 9.451), Acc: 0.795, MRR: 0.848(0.829, 0.868) with dbsearch size: 80000
Mean rank: 8.747(6.008, 11.486), Acc: 0.784, MRR: 0.838(0.818, 0.858) with dbsearch size: 100000
CPU times: user 10min 36s, sys: 30min 23s, total: 41min
Wall time: 10min 20s
